In [1]:
import json
import pandas as pd 
import glob
from pathlib import Path
import os


In [2]:
def get_columns_name(schemas,ds_name,sorting_key='column_position'):
    column_details=schemas[ds_name]
    columns=sorted(column_details,key=lambda col:col[sorting_key])
    return [col['column_name'] for col in columns]

In [3]:
def read_csv(file,schemas):
    file_path_list=Path(file).parts
    ds_name=file_path_list[-2]
    columns=get_columns_name(schemas,ds_name)
    df=pd.read_csv(file,names=columns)
    return df


In [4]:
def to_json(df,tgt_base_dir,ds_name,file_name):
    json_file_path=f'{tgt_base_dir}/{ds_name}/{file_name}'
    os.makedirs(f'{tgt_base_dir}/{ds_name}')
    df.to_json(
        json_file_path,
        orient='records',
        lines=True
    )


In [5]:
def file_converter(src_base_dir,tgt_base_dir,ds_name):
    files=glob.glob(f'{src_base_dir}/{ds_name}/part-*')
    schemas=json.load(open(f'{src_base_dir}/schemas.json'))
    for file in files:
        df=read_csv(file,schemas)
        file_name=Path(file).parts [-1]
        to_json(df,tgt_base_dir,ds_name,file_name)


In [6]:
def process_files(ds_names=None):
    src_base_dir='data/retail_db'
    tgt_base_dir='data/retail_db_json'
    schemas=json.load(open(f'{src_base_dir}/schemas.json'))
    
    if not ds_names:
        ds_names=schemas.keys()
    for ds_name in ds_names:
        print(f'Processing {ds_name}')
        file_converter(src_base_dir, tgt_base_dir, ds_name)


In [7]:
process_files()

Processing departments
Processing categories
Processing orders
Processing products
Processing customers
Processing order_items
